#THE FIRST PROBLEM:-

In [44]:
#importing useful libraries & cheking the pandas version
import numpy as np
import pandas as pd
pd.__version__

'1.0.5'

In [45]:
#reading user .csv file using pandas
df=pd.read_csv("/content/drive/My Drive/Lannet_solns (1).csv")


In [46]:
#checking the datatypes of columns
df.dtypes

name           object
buy_date       object
sell_date      object
buy_price       int64
sell_price      int64
distributor    object
MFG            object
dtype: object

In [47]:
#printing first 5 rows of the dataframe
print(df.head())

     name     buy_date  ... distributor             MFG
0     Kim  27 May,2010  ...          BD             NaN
1    John   03-02-2010  ...          AD      27-08-2008
2    Nobi   02-09-2012  ...         PqD      033-7-1998
3    Ravi   06/15/2012  ...          MD       6/17/2010
4  Pankaj   17-06-2014  ...        C&CD  August-2012-19

[5 rows x 7 columns]


In [48]:
#function to get the date columns only
def date_find(df):
  #taking a copy of orignal dataframe so that the original one remains intact
  df1=df.copy()
  #using a for loop to check whether a column is convertable to pandas date format
  for col in df1.columns:
    if df1[col].dtype == 'object':
        try:
          #converting object type columns into date (datetime64[ns]) type & invalid or null values to NaT
            df1[col] = pd.to_datetime(df1[col],errors='coerce')
        except ValueError:
            pass
  #if any column contains all NaT (null/invalid) values; i.e. not date columns, drop them          
  df1=df1.dropna(how='all',axis=1)
  #return only the columns containing date
  return df1.select_dtypes(include='datetime64[ns]')



In [49]:
#output of the function(only date columns)
print(date_find(df))

     buy_date  sell_date        MFG
0  2010-05-27 2010-09-15        NaT
1  2010-03-02 2010-07-23 2008-08-27
2  2012-02-09 2012-09-26        NaT
3  2012-06-15 2012-08-10 2010-06-17
4  2014-06-17 2015-01-12 2012-08-19
5  2010-05-23 2010-06-14 2009-07-13
6  2014-08-15 2014-09-25 2012-01-19
7  2011-04-10 2012-01-01 2010-01-01
8  2012-01-07        NaT 2013-01-01
9  2008-02-16 2018-09-19        NaT
10 2012-07-13        NaT 2010-09-17
11 2012-03-09 2012-12-14 2018-07-13
12 2013-06-06 2013-09-12        NaT
13 2013-08-02 2013-09-06 2012-05-23


In [50]:
#Creating new columns with the difference of dates:-
data=date_find(df)
def difference(data):
  a = data.values
  r,c = np.triu_indices(a.shape[1],1)
  cols = data.columns
  nm = [cols[i]+"_"+cols[j] for i,j in zip(r,c)]
  return pd.DataFrame(a[:,r] - a[:,c], columns=nm)




In [51]:
#output of the function(only the difference of date coloumns as new columns)
print(difference(data))

   buy_date_sell_date buy_date_MFG sell_date_MFG
0           -111 days          NaT           NaT
1           -143 days     552 days      695 days
2           -230 days          NaT           NaT
3            -56 days     729 days      785 days
4           -209 days     667 days      876 days
5            -22 days     314 days      336 days
6            -41 days     939 days      980 days
7           -266 days     464 days      730 days
8                 NaT    -360 days           NaT
9          -3868 days          NaT           NaT
10                NaT     665 days           NaT
11          -280 days   -2317 days    -2037 days
12           -98 days          NaT           NaT
13           -35 days     436 days      471 days


#Solution of the complete problem with a single function(final solution)



In [52]:
#Solution of the complete problem with a single function(final solution)
def Date_diff(df):
  df1=df.copy()
  for col in df1.columns:
    if df1[col].dtype == 'object':
        try:
            df1[col] = pd.to_datetime(df1[col],errors='coerce')
        except ValueError:
            pass
  df1=df1.dropna(how='all',axis=1)

  data=df1.select_dtypes(include='datetime64[ns]')         
  a = data.values
  r,c = np.triu_indices(a.shape[1],1)
  cols = data.columns
  nm = [cols[i]+"_"+cols[j] for i,j in zip(r,c)]
  temp=pd.DataFrame(a[:,r] - a[:,c], columns=nm)
  #Joining new columns (differences) to the original dataframe
  df = pd.concat([df, temp], axis=1)
  return df


In [53]:
#Final modified dataframe(Final Output)
print(Date_diff(df))

      name          buy_date  ... buy_date_MFG  sell_date_MFG
0      Kim       27 May,2010  ...          NaT            NaT
1     John        03-02-2010  ...     552 days       695 days
2     Nobi        02-09-2012  ...          NaT            NaT
3     Ravi        06/15/2012  ...     729 days       785 days
4   Pankaj        17-06-2014  ...     667 days       876 days
5    Subha        23-05-2010  ...     314 days       336 days
6    Dyana  15th August-2014  ...     939 days       980 days
7    Kabir      Apr 10, 2011  ...     464 days       730 days
8   Sahnku         07-Jan-12  ...    -360 days            NaT
9    Bimal  February/16/2008  ...          NaT            NaT
10   Richi        07-13-2012  ...     665 days            NaT
11  Ayushi        03-09-2012  ...   -2317 days     -2037 days
12   Mohit        06-06-2013  ...          NaT            NaT
13   Krish        08-02-2013  ...     436 days       471 days

[14 rows x 10 columns]


In [54]:
#time taken in executing the function!
%timeit Date_diff(df)

100 loops, best of 3: 11.3 ms per loop


#The Second Problem:-


In [55]:
import pandas as pd
import numpy as np
from scipy import stats

In [56]:
#storing the .csv (taken from Kaggle) file as a pandas daframe
ds=pd.read_csv('/content/drive/My Drive/datasets_26073_33239_weight-height.csv')

In [57]:
#printing the top rows of the dataset
print(ds.head())

  Gender     Height      Weight
0   Male  73.847017  241.893563
1   Male  68.781904  162.310473
2   Male  74.110105  212.740856
3   Male  71.730978  220.042470
4   Male  69.881796  206.349801


In [58]:
#checking the number of rows and columns
ds.shape

(10000, 3)

In [59]:
#This is the function(see in comment)
def detect_drop_outliers(df):
  z_ideal=3
  # Constrains will contain `True` or `False` depending on z_ideal.
  constrains = df.select_dtypes(include=[np.number]) \
  .apply(lambda x: np.abs(stats.zscore(x)) < z_ideal, result_type='reduce') \
  .all(axis=1)
  # Drop Rows with incorrect entry in any one or more columns.
  df.drop(df.index[~constrains], inplace=True)
  return df

In [60]:
#Output Dataset Shape, Clearly 7 rows are removed.
detect_drop_outliers(ds).shape

(9993, 3)

In [61]:
#time taken in executing the function!
%timeit detect_drop_outliers(ds)

100 loops, best of 3: 4.68 ms per loop


In [62]:
#storing a new large dataset to test my function
new=pd.read_csv('/content/drive/My Drive/bhp.csv')

In [63]:
new.shape

(13200, 7)

In [64]:
detect_drop_outliers(new).shape

(12675, 7)

In [65]:
#Justifying my function performance 
sample=pd.read_csv('/content/drive/My Drive/solution (6).csv')

In [66]:
sample.shape

(11, 7)

In [67]:
print(sample)

    Dsg.          Name  age   salary     PF              Job         DOB
0    Mr.     Sujit Som   37    14000   1500     Project Mng.  27-09-1983
1    Mr.    Mohan Giri   45    17500   1750     Project Mng.  01-05-1986
2   Mrs.  Bidipta Basu   43    21000  21000     Project Mng.  23-07-1977
3    Mr.   Subha Kumar   14    16000   1500       Accountant  15-01-1975
4    Ms.     Richa Sen   39    19000    190        Developer  03-06-1903
5   Mrs.     Soumi Dey   52    16500   1650     Project Mng.  05-05-2068
6    Mr.  Kumar Mahato   50    25000   2500       Archietect  05-02-1970
7    Ms.     Moumi Roy  110  1000000   1000  Junior Engineer  30-06-1980
8    Ms.      Margaret   41    40000   4000   Ast. Secretory  01-01-1979
9    Mr.   Karan Singh   38    15000   1500        Developer  12-12-1982
10  Mrs.    Rimi Ahuja   50    50000   5000        Secretory  17-07-1970


In [68]:
detect_drop_outliers(sample).shape

(9, 7)

In [69]:
#See comment please
detect_drop_outliers(sample)

,Dsg.,Name,age,salary,PF,Job,DOB
0,Mr.,Sujit Som,37,14000,1500,Project Mng.,27-09-1983
1,Mr.,Mohan Giri,45,17500,1750,Project Mng.,01-05-1986
3,Mr.,Subha Kumar,14,16000,1500,Accountant,15-01-1975
4,Ms.,Richa Sen,39,19000,190,Developer,03-06-1903
5,Mrs.,Soumi Dey,52,16500,1650,Project Mng.,05-05-2068
6,Mr.,Kumar Mahato,50,25000,2500,Archietect,05-02-1970
8,Ms.,Margaret,41,40000,4000,Ast. Secretory,01-01-1979
9,Mr.,Karan Singh,38,15000,1500,Developer,12-12-1982
10,Mrs.,Rimi Ahuja,50,50000,5000,Secretory,17-07-1970


#Final Solution Function(Second Problem):-

In [70]:
#Here is the desired function
def detect_drop_outliers(df):
  z_ideal=3
  # Constrains will contain `True` or `False` depending on z_ideal.
  constrains = df.select_dtypes(include=[np.number]) \
  .apply(lambda x: np.abs(stats.zscore(x)) < z_ideal, result_type='reduce') \
  .all(axis=1)
  # Drop incorrect entries (rows)
  df.drop(df.index[~constrains], inplace=True)
  return df

#The Third Problem:-

In [71]:
#importing python libraries
import numpy as np
import pandas as pd

In [72]:
#loading the .csv file into pandas dataframe
Set=pd.read_csv('/content/drive/My Drive/bhp.csv')

In [73]:
#checking the shape of the dataset
Set.shape

(13200, 7)

In [74]:
#checking the top rows
Set.head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250


In [75]:
#The function which will give the desired solution
def pearson_corr(df):
  #separating the continuous and categorical columns
  df_con=df.select_dtypes(include=[np.number])
  df_cat=df.select_dtypes(exclude=[np.number])
  #setting the threshold correlation value as given in the question
  threshold=0.85
  #finding the Pearson correlation between the values of continuous columns 
  df_corr = df_con.corr(method='pearson', min_periods=1)
  #now detecting the correlation values higher than the threshold
  df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
  #removing the columns with high correlated values
  un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
  #setting the structured/edited columns into the output dataframe
  df_out = df_con[un_corr_idx]
  #Joining the catagorical columns to get the exact desired output
  df_out =pd.concat([df_cat,df_out],axis=1)
  return df_out

In [76]:
#applying our dataset in our function
pearson_corr(Set)

,location,size,total_sqft,bath,price,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,3699
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,4305
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,6245
4,Kothanur,2 BHK,1200.0,2.0,51.00,4250
...,...,...,...,...,...,...
13195,Whitefield,5 Bedroom,3453.0,4.0,231.00,6689
13196,other,4 BHK,3600.0,5.0,400.00,11111
13197,Raja Rajeshwari Nagar,2 BHK,1141.0,2.0,60.00,5258
13198,Padmanabhanagar,4 BHK,4689.0,4.0,488.00,10407


In [77]:
#loading a comparatively large and complex dataset to test our function
test=pd.read_csv('/content/drive/My Drive/2018_Financial_Data.csv')

In [78]:
#checking the dataset
test

,Unnamed: 0,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,Income Tax Expense,Net Income - Non-Controlling int,Net Income - Discontinued ops,Net Income,Preferred Dividends,Net Income Com,EPS,EPS Diluted,Weighted Average Shs Out,Weighted Average Shs Out (Dil),Dividend per Share,Gross Margin,EBITDA Margin,EBIT Margin,Profit Margin,Free Cash Flow margin,EBITDA,EBIT,Consolidated Income,Earnings Before Tax Margin,Net Profit Margin,Cash and cash equivalents,Short-term investments,Cash and short-term investments,Receivables,Inventories,Total current assets,"Property, Plant & Equipment Net",Goodwill and Intangible Assets,...,Payables Turnover,Inventory Turnover,ROE,Capex per Share,Gross Profit Growth,EBIT Growth,Operating Income Growth,Net Income Growth,EPS Growth,EPS Diluted Growth,Weighted Average Shares Growth,Weighted Average Shares Diluted Growth,Dividends per Share Growth,Operating Cash Flow growth,Free Cash Flow growth,10Y Revenue Growth (per Share),5Y Revenue Growth (per Share),3Y Revenue Growth (per Share),10Y Operating CF Growth (per Share),5Y Operating CF Growth (per Share),3Y Operating CF Growth (per Share),10Y Net Income Growth (per Share),5Y Net Income Growth (per Share),3Y Net Income Growth (per Share),10Y Shareholders Equity Growth (per Share),5Y Shareholders Equity Growth (per Share),3Y Shareholders Equity Growth (per Share),10Y Dividend per Share Growth (per Share),5Y Dividend per Share Growth (per Share),3Y Dividend per Share Growth (per Share),Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Sector,2019 PRICE VAR [%],Class
0,CMCSA,9.450700e+10,0.1115,0.000000e+00,9.450700e+10,0.000000e+00,6.482200e+10,7.549800e+10,1.900900e+10,3.542000e+09,1.511100e+10,3.380000e+09,131000000.0,0.000000e+00,1.173100e+10,0.0,1.173100e+10,2.56,2.53,4.549505e+09,4.549505e+09,0.917,1.0000,0.310,0.1974,0.124,0.1268,2.932900e+10,1.865300e+10,1.186200e+10,0.1599,0.1241,3.814000e+09,0.000000e+00,3.814000e+09,1.110400e+10,0.000000e+00,2.184800e+10,4.443700e+10,1.676230e+11,...,3.6159,0.0000,0.1638,-2.7062,0.1115,0.0220,0.0550,-0.4840,-0.4700,-0.4674,-0.0266,-0.0305,0.9387,0.1428,0.2462,0.1327,0.1094,0.1107,0.1164,0.1456,0.1044,0.1928,0.1463,0.1579,0.0841,0.1019,0.1396,0.2558,0.1865,0.2348,0.2570,0.0000,0.3426,0.0722,0.7309,0.0000,0.1308,Consumer Cyclical,32.794573,1
1,KMI,1.414400e+10,0.0320,7.288000e+09,6.856000e+09,0.000000e+00,6.010000e+08,3.062000e+09,3.794000e+09,1.917000e+09,2.196000e+09,5.870000e+08,310000000.0,0.000000e+00,1.609000e+09,128000000.0,1.481000e+09,0.66,0.66,2.207018e+09,2.216000e+09,0.725,0.4847,0.453,0.2908,0.105,0.1498,6.410000e+09,4.113000e+09,1.919000e+09,0.1553,0.1138,3.280000e+09,0.000000e+00,3.280000e+09,1.498000e+09,3.850000e+08,5.722000e+09,3.789700e+10,2.484500e+10,...,2.8248,34.9666,0.0478,-1.3195,0.0564,0.0405,0.0751,53.8519,65.0000,65.0000,-0.0063,-0.0063,0.4500,0.0961,0.3841,NaN,-0.1402,-0.0104,NaN,-0.1057,-0.0215,NaN,-0.0881,0.8445,NaN,0.0376,-0.0182,0.0000,-0.1421,-0.2785,0.0345,-0.0920,-0.0024,0.0076,-0.0137,0.0000,-0.1265,Energy,40.588068,1
2,INTC,7.084800e+10,0.1289,2.711100e+10,4.373700e+10,1.354300e+10,6.750000e+09,2.042100e+10,2.331600e+10,-1.260000e+08,2.331700e+10,2.264000e+09,0.0,0.000000e+00,2.105300e+10,0.0,2.105300e+10,4.57,4.48,4.564000e+09,4.611000e+09,1.200,0.6173,0.456,0.3273,0.297,0.2011,3.227600e+10,2.319100e+10,2.105300e+10,0.3291,0.2972,3.019000e+09,8.631000e+09,1.165000e+10,6.722000e+09,7.253000e+09,2.878700e+10,4.897600e+10,3.634900e+10,...,5.5264,9.9534,0.2824,-3.2923,0.1187,0.1203,0.2917,1.1928,1.2402,1.2513,-0.0191,-0.0277,0.1132,0.3312,0.3793,0.0876,0.0770,0.0959,0.1271,0.0883,0.1675,0.1719,0.1872,0.2377,0.0876,0.0665,0.0787,0.0815,0.0592,0.0772,0.1989,0.0387,0.0382,0.1014,-0.0169,0.0390,-0.0942,Technology,30.295514,1
3,MU,3.039100e+10,0.4955,1.250000e+10,1.789100e+10,2.141000e+09,8.130000e+08,2.897

In [79]:
#Checking how many correlations pairs have corr. more than 0.85
#finding absolute correlation values
corr = test.corr(method='pearson',min_periods=1).abs()

# setting equality (self correlation) as zero
corr[corr == 1] = 0

#unstack the column-correlations
corr_cols = corr.unstack()

# displaying the highly correlated features
display(corr_cols[corr_cols>.85])

Revenue                                 Cost of Revenue                           0.953780
Revenue Growth                          Gross Profit Growth                       0.999605
Cost of Revenue                         Revenue                                   0.953780
Gross Profit                            SG&A Expense                              0.850023
                                        Operating Expenses                        0.954962
                                                                                    ...   
Weighted Average Shares Diluted Growth  Asset Growth                              0.912485
Free Cash Flow growth                   operatingCycle                            0.871458
Receivables growth                      daysOfInventoryOutstanding                0.995976
                                        Days Sales Outstanding                    0.868559
Asset Growth                            Weighted Average Shares Diluted Growth    0.912485

In [80]:
#applying the function
pearson_corr(test)

,Unnamed: 0,Sector,Revenue,Revenue Growth,Gross Profit,R&D Expenses,Operating Income,Interest Expense,Income Tax Expense,Net Income - Non-Controlling int,Net Income - Discontinued ops,Preferred Dividends,EPS,Weighted Average Shs Out,Dividend per Share,Gross Margin,EBITDA Margin,EBIT Margin,Cash and cash equivalents,Short-term investments,Cash and short-term investments,Receivables,Inventories,"Property, Plant & Equipment Net",Goodwill and Intangible Assets,Long-term investments,Tax assets,Payables,Short-term debt,Deferred revenue,Tax Liabilities,Other comprehensive income,Retained earnings (deficit),Net Debt,Other Assets,Stock-based compensation,Operating Cash Flow,Acquisitions and disposals,Investment purchases and sales,Investing Cash flow,...,dividendPayoutRatio,Revenue per Share,Market Cap,Earnings Yield,Income Quality,Dividend Yield,Intangibles to Total Assets,Graham Net-Net,Working Capital,Average Payables,Receivables Turnover,EBIT Growth,Net Income Growth,EPS Growth,EPS Diluted Growth,Weighted Average Shares Growth,Dividends per Share Growth,Operating Cash Flow growth,10Y Revenue Growth (per Share),5Y Revenue Growth (per Share),3Y Revenue Growth (per Share),10Y Operating CF Growth (per Share),5Y Operating CF Growth (per Share),3Y Operating CF Growth (per Share),10Y Net Income Growth (per Share),5Y Net Income Growth (per Share),3Y Net Income Growth (per Share),10Y Shareholders Equity Growth (per Share),5Y Shareholders Equity Growth (per Share),3Y Shareholders Equity Growth (per Share),10Y Dividend per Share Growth (per Share),5Y Dividend per Share Growth (per Share),3Y Dividend per Share Growth (per Share),Inventory Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,2019 PRICE VAR [%],Class
0,CMCSA,Consumer Cyclical,9.450700e+10,0.1115,9.450700e+10,0.000000e+00,1.900900e+10,3.542000e+09,3.380000e+09,131000000.0,0.000000e+00,0.0,2.56,4.549505e+09,0.917,1.0000,0.310,0.1974,3.814000e+09,0.000000e+00,3.814000e+09,1.110400e+10,0.000000e+00,4.443700e+10,1.676230e+11,7.883000e+09,0.000000e+00,8.494000e+09,4.398000e+09,2.182000e+09,2.758900e+10,-3.680000e+08,4.198300e+10,1.079290e+11,6.930000e+09,8.260000e+08,2.429700e+10,-3.807800e+10,-1.257000e+09,-5.085400e+10,...,0.285739,20.7730,1.549106e+11,0.0752,2.0712,0.0269,0.6660,-1.0156,-5.755000e+09,7.701000e+09,9.4801,0.0220,-0.4840,-0.4700,-0.4674,-0.0266,0.9387,0.1428,0.1327,0.1094,0.1107,0.1164,0.1456,0.1044,0.1928,0.1463,0.1579,0.0841,0.1019,0.1396,0.2558,0.1865,0.2348,0.0000,0.0722,0.7309,0.0000,0.1308,32.794573,1
1,KMI,Energy,1.414400e+10,0.0320,6.856000e+09,0.000000e+00,3.794000e+09,1.917000e+09,5.870000e+08,310000000.0,0.000000e+00,128000000.0,0.66,2.207018e+09,0.725,0.4847,0.453,0.2908,3.280000e+09,0.000000e+00,3.280000e+09,1.498000e+09,3.850000e+08,3.789700e+10,2.484500e+10,7.481000e+09,1.566000e+09,1.337000e+09,3.388000e+09,0.000000e+00,4.830000e+08,-3.300000e+08,-7.716000e+09,3.404400e+10,5.590000e+08,0.000000e+00,5.043000e+09,2.928000e+09,-7.200000e+07,-6.800000e+07,...,1.005594,6.3827,3.394394e+10,0.0429,3.4051,0.0471,0.3150,-1.1179,-1.835000e+09,1.338500e+09,9.6022,0.0405,53.8519,65.0000,65.0000,-0.0063,0.4500,0.0961,NaN,-0.1402,-0.0104,NaN,-0.1057,-0.0215,NaN,-0.0881,0.8445,NaN,0.0376,-0.0182,0.0000,-0.1421,-0.2785,-0.0920,0.0076,-0.0137,0.0000,-0.1265,40.588068,1
2,INTC,Technology,7.084800e+10,0.1289,4.373700e+10,1.354300e+10,2.331600e+10,-1.260000e+08,2.264000e+09,0.0,0.000000e+00,0.0,4.57,4.564000e+09,1.200,0.6173,0.456,0.3273,3.019000e+09,8.631000e+09,1.165000e+10,6.722000e+09,7.253000e+09,4.897600e+10,3.634900e+10,9.430000e+09,0.000000e+00,3.824000e+09,1.261000e+09,0.000000e+00,6.562000e+09,-9.740000e+08,5.017200e+10,1.470900e+10,3.162000e+09,1.546000e+09,2.943200e+10,3.580000e+08,3.856000e+09,-1.123900e+10,...,0.263193,15.3650,2.141885e+11,0.0974,1.3980,0.0256,0.2841,-0.1149,1.216100e+10,3.376000e+09,11.4929,0.1203,1.1928,1.2402,1.2513,-0.0191,0.1132,0.3312,0.0876,0.0770,0.0959,0.1271,0.0883,0.1675,0.1719,0.1872,0.2377,0.087

In [82]:
#Cheking the time taken by the function
%timeit pearson_corr(df)

100 loops, best of 3: 5.14 ms per loop
